Routines used to loop through image lists and print out information.

Also includes a routine to check for saturation, but need to have pixel mask (which can be made in Data_reduction_pipeline.ipynb) and need to analyze raw data.

In [ ]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.ndimage import filters

In [ ]:
# Directory and list of images to examine. Generally, image list here is all raw images in a given directory.
# To make image lists, pipe output of ls into a file (e.g. ls *FIT > imlist)

directory = '/Users/dahlek/Desktop/2017-03-26/' 
jupiter = np.loadtxt(directory+'imlist',dtype=str)

In [ ]:
# Print given keywords of interest for each file. Here, I'm printing the name of the file, whether that image is RF-on (1) or RF-off (0), the wavelength, the exposure time, and the UT date it was recorded
# This allows the user to find the beginning and end of a given datacube - this information can be used below to copy and paste a cube list into a new text file.

for i in range(0,len(jupiter)):
    im = fits.open(directory+jupiter[i])
    print jupiter[i],im[0].header['rfon'],im[0].header['lambda'],im[0].header['exposure'],im[0].header['date'],im[0].header['object']
    im.close()

In [ ]:
# print out same image list, but without extra information. copy and paste collection of files of interest from this list.
for i in range(0,len(jupiter)):
    im = fits.open(directory+jupiter[i])
    print jupiter[i]
    im.close()

In [ ]:
# Once list is saved, check list for repeat wavelengths
# If there aren't any repeat wavelenths, will return set()
# If there are repeat wavelengths, be sure to comment them out. Generally, repeat wavelengths arise after restarting a section of cube due to over-saturation, so it's generally safer to comment out the longer exposure times (especially if the following repeat wavelengths were imaged at a shorter exposure time)

wl = []
p = 0
# whatever or wherever you saved the new image list
directory = '/Users/dahlek/Desktop/march2017_2/jupiter5/' 
jupiter = np.loadtxt('/Users/dahlek/Desktop/march2017_2/'+'juplist5',dtype=str) 
for i in range(0,len(jupiter)):
    im = fits.open(directory+jupiter[i])
    if im[0].header['rfon'] == 1:
        wl.append(im[0].header['lambda'])
        #print jupiter[i],im[0].header['rfon'],im[0].header['lambda'],im[0].header['exposure'],im[0].header['date'],p
        p += 1
    im.close()
    
set([x for x in wl if wl.count(x) > 1])

In [ ]:
# Same routine as Jupiter lists, but for stars:

In [ ]:
#making star lists
for i in range(0,len(stars)):
    im = fits.open(directory+'/stars/'+stars[i])
    #if im[0].header['lambda'] > 950 and im[0].header['rfon'] == 1:
    print stars[i],im[0].header['rfon'],im[0].header['lambda'],im[0].header['exposure'],im[0].header['ut'],im[0].header['object']
    im.close()

In [ ]:
for i in range(0,len(stars)):
    im = fits.open(directory+'/stars/'+stars[i])
    #if im[0].header['lambda'] > 950 and im[0].header['rfon'] == 1:
    print stars[i]
    im.close()

In [ ]:
# check for repeat wavelengths. For star cubes, sometimes it's nice to keep as many wavelengths as possible bc of photometric fits later
wl = []
for i in range(0,len(stars)):
    im = fits.open(directory+stars[i])
    if im[0].header['rfon'] == 1:
        wl.append(im[0].header['lambda'])
        print stars[i],im[0].header['orig_wl'],im[0].header['exposure'],im[0].header['ut'],im[0].header['object']
    
set([x for x in wl if wl.count(x) > 1])

In [ ]:
# Sometimes it's useful to visualize the wavelengths over the course of a cube
plt.plot(wl,'.')
plt.show()

In [ ]:
# plot some raw data if you feel like it
plt.matshow(fits.open(directory+jupiter[15])[0].data)
plt.matshow(fits.open(directory+jupiter[150])[0].data)
plt.show()

In [ ]:
# flats
directory = '/Users/dahlek/Desktop/march2017/flats/'
imlist = np.loadtxt('/Users/dahlek/Desktop/march2017/flatlist',dtype=str)
for i in range(0,len(imlist)):
    im = fits.open(directory+imlist[i])
    print imlist[i],im[0].header['lambda'],im[0].header['rfon']

In [ ]:
im = fits.open('/Users/dahlek/Desktop/march2017/jupiter/NC0000179196.FIT')
plt.matshow(im[0].data)
plt.colorbar()
plt.show()

In [ ]:
imfringe = fits.open('/Users/dahlek/Desktop/March_2017_data_Synthetic_fringe_files/Jupiter/NC0000179196.FIT')
plt.matshow(imfringe[0].data)
plt.colorbar()
plt.show()

In [ ]:
plt.matshow(im[0].data/imfringe[0].data)
plt.colorbar()
plt.show()

In [ ]:
def sat_check(directory,images,pixels):
    
    #images = '/home/users/dahlek/NAIC/cube_lists/feb_2017' #*location and list of images to check*
    #directory = '/zenith-data/dahlek/raw_data/2017-02-02/' #*data directory where fits files are*
    #pixels = '/zenith-data/dahlek/' #location and text file of pixel mask.*
    
    #sat_value = 43836. #from linearity_tests.ipynb VALUE FOR OLD CAMERA, PRE-JAN 2018 DATA!
    #sat_value = 27073. #3-sigma VALUE FOR NEW CAMERA
    #sat_value = 30000. #test value
    sat_value = 30400 #5 sigma value for new camera
    max_pixels_rfon = 6000 #from Paul's thesis
    max_pixels_rfoff = 10 #from Paul's thesis

    sat_images_rfon = []
    sat_images_rfoff = []
    number_of_rfoffs = []
    number_of_rfons = []
    unbinned = []

    imlist = np.loadtxt(images,dtype=str) #load the images to look at
    pixel_mask = np.loadtxt(pixels,dtype=int) #Array of positions of hot pixels (Don't count)

    for i in range(0,len(imlist)):

        sat_pix = []
        im = fits.open(directory+imlist[i])
        data = im[0].data
        header = im[0].header
        rf = header['rfon']
        if len(data) == 1024:
            unbinned.append(1)
            continue

        if rf == 1: #If RF is on, check if there are more than 6000 nonlinear pixels. 
            #Right here, put qualifiers for being within a certain position on the sky
            number_of_rfons.append(1)
            for x in range(0,len(im[0].data)):
                for j in range(0,len(im[0].data[x])):
                    #Qualifier to ignore bad pixels:
                    if pixel_mask[x][j] == 1:
                        continue
                    #If it's not a hot pixel, is it greater than the sat_value? If so, append a count
                    elif im[0].data[x][j] > sat_value:
                        sat_pix.append(1)       
            if len(sat_pix) >= max_pixels_rfon:
                #print 'There are more than 10 saturated pixels in',imlist[i]
                sat_images_rfon.append(imlist[i])


        else: #RF is off
            number_of_rfoffs.append(1)
            for x in range(0,len(im[0].data)):
                for j in range(0,len(im[0].data[x])):
                    #Qualifier to ignore bad pixels:
                    if pixel_mask[x][j] == 1:
                        continue
                    #If it's not a hot pixel, is it greater than the sat_value? If so, append a count
                    elif im[0].data[x][j] > sat_value:
                        sat_pix.append(1)
            if len(sat_pix) >= max_pixels_rfoff:
                sat_images_rfoff.append(imlist[i])

        #Save number of sat pixels to header
        header['SAT_VAL'] = (sat_value,'Saturation level (DN)')
        header['SAT_PIX'] = (len(sat_pix),'Number of pixels above saturation level')
        im.writeto(directory+imlist[i],overwrite=True)
        im.close()


    print 'There were '+str(len(imlist))+' images total in '+directory
    print 'There were '+str(len(sat_images_rfoff))+' saturated RF-off images:'
    print sat_images_rfoff
    print 'There were '+str(len(sat_images_rfon))+' saturated RF-on images:'
    print sat_images_rfon
    print 'There were',str(len(unbinned)),'unbinned images that weren\'t analyzed.'